In [2]:
#importing libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
#loading data
dataset_train = pd.read_csv(r"C:\Users\LENOVO\Desktop\Major and Minore projects\stock price dataset.csv")
dataset_train.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
1,2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2,2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
3,2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
4,2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900


In [4]:
training_set = dataset_train.iloc[:,1:2]
print(training_set)
print(training_set.shape)



            Open
0     262.000000
1     247.699997
2     266.579987
3     267.079987
4     253.850006
...          ...
1004  401.970001
1005  432.959991
1006  448.250000
1007  421.440002
1008  407.309998

[1009 rows x 1 columns]
(1009, 1)


In [5]:
#normalizing the dataset
from sklearn.preprocessing import MinMaxScaler

scaler =MinMaxScaler(feature_range = (0,1))
scaled_training_set = scaler.fit_transform(training_set)
scaled_training_set

array([[0.06125254],
       [0.03005911],
       [0.07124314],
       ...,
       [0.46753051],
       [0.4090483 ],
       [0.3782257 ]])

In [6]:
#Creating X_train and Y_train Data Structure

X_train = []
Y_train = []
for i in range(60,1008):
    X_train.append(scaled_training_set[i-60:i,0])
    Y_train.append(scaled_training_set[i , 0])
X_train = np.array(X_train)
Y_train = np.array(Y_train)


In [7]:
print(X_train.shape)
print(Y_train.shape)

(948, 60)
(948,)


In [8]:
#REshaping the Data

X_train = np.reshape(X_train,(X_train.shape[0], X_train.shape[1],1))
X_train.shape


(948, 60, 1)

In [9]:
#Building the model by importing the crucial libraries and adding different layers to LSTM

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout


In [10]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))


In [ ]:
#Fitting the model

regressor.compile(optimizer = 'adam',loss = 'mean_squared_error')
regressor.fit(X_train, Y_train, epochs = 100, batch_size=32)

Epoch 1/100
30/30 [==============================] - 92s 866ms/step - loss: 0.0392
Epoch 2/100
30/30 [==============================] - 23s 759ms/step - loss: 0.0098
Epoch 3/100
30/30 [==============================] - 34s 1s/step - loss: 0.0082
Epoch 4/100
30/30 [==============================] - 34s 1s/step - loss: 0.0071
Epoch 5/100
30/30 [==============================] - 50s 2s/step - loss: 0.0072
Epoch 6/100
30/30 [==============================] - 48s 2s/step - loss: 0.0075
Epoch 7/100
30/30 [==============================] - 27s 865ms/step - loss: 0.0064
Epoch 8/100
30/30 [==============================] - 21s 683ms/step - loss: 0.0068
Epoch 9/100
30/30 [==============================] - 20s 672ms/step - loss: 0.0068
Epoch 10/100
30/30 [==============================] - 21s 685ms/step - loss: 0.0072
Epoch 11/100
30/30 [==============================] - 21s 688ms/step - loss: 0.0058
Epoch 12/100
30/30 [==============================] - 21s 699ms/step - loss: 0.0056
Epoch 13/100


In [ ]:
#Extracting the actual stock price 

dataset_test = pd.read_csv(r"C:\Users\LENOVO\Desktop\Major and Minore projects\stock price dataset.csv")
actual_stock_price = dataset_test.iloc[:,1:2].values

In [ ]:
#preparing the input for the model

dataset_total = pd.concat((dataset_train['Open'],dataset_test['Open']),axis = 0)
inputs = dataset_total[len(dataset_total)-len(dataset_test)-60:].values

inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)

X_test = []
for i in range(60,80):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0], X_test.shape[1],1))
    

In [ ]:
#predicting stock price

predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)



In [ ]:
#Plotting the actual and predicted priced

plt.plot(actual_stock_price, color = 'red', label = 'Actual Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()